In [4]:
# ==== CS-340 Project Two Dashboard (final) ====
# Header: larger logo, title with the same logo mirrored on the right.
# Table: compact logo column painted via CSS (Codio-safe; no markdown).

from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()

import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64, os, sys

# Allow importing the CRUD module in the same folder
sys.path.append(os.getcwd())
from CRUD_Python_Module import AnimalShelter


# -----------------------
# Logo setup
# -----------------------
LOGO_FILENAME = "Grazioso Salvare Logo.png"  # change only if your file name differs
LOGO_PATH = os.path.join(os.getcwd(), LOGO_FILENAME)
LOGO_HEADER_HEIGHT = 68  # <-- a little bigger in the header

logo_uri = None
if os.path.exists(LOGO_PATH):
    with open(LOGO_PATH, "rb") as f:
        logo_uri = "data:image/png;base64," + base64.b64encode(f.read()).decode("utf-8")


# -----------------------
# Data / Model
# -----------------------
# Local Mongo in Codio; collection = animals in db aac
db = AnimalShelter(uri="mongodb://127.0.0.1:27017", db_name="aac", collection_name="animals")

def fetch_df(query=None, projection=None, limit=0) -> pd.DataFrame:
    """Read via CRUD, unwrap to list, DataFrame, drop _id, add 'logo' column (text blank; CSS paints image)."""
    result = db.read(query or {}, projection or {}, limit)
    data = result.detail if getattr(result, "ok", False) and isinstance(result.detail, list) else []
    df_ = pd.DataFrame.from_records(data)
    if "_id" in df_.columns:
        df_.drop(columns=["_id"], inplace=True)
    # First column reserved for the logo icon
    if "logo" not in df_.columns:
        df_.insert(0, "logo", "")
    return df_

# initial data
df = fetch_df({}, limit=1000)
print("Initial rows loaded:", len(df))

DEFAULT_COLS = ['name','breed','animal_type','age_upon_outcome','sex_upon_outcome']
cols_for_table = list(df.columns) if not df.empty else (["logo"] + DEFAULT_COLS)


# -----------------------
# App / Layout
# -----------------------
app = JupyterDash(__name__)

# Header with bigger logo on left AND right of the title
header_children = []
if logo_uri:
    header_children.append(html.Img(src=logo_uri, style={"height": f"{LOGO_HEADER_HEIGHT}px"}))
header_children.append(html.H1("CS-340 Dashboard", style={"margin": 0}))
if logo_uri:
    header_children.append(html.Img(src=logo_uri, style={"height": f"{LOGO_HEADER_HEIGHT}px"}))

header = html.Div(
    header_children,
    style={"display": "flex", "alignItems": "center", "justifyContent": "center", "gap": "14px"}
)

# Datatable columns (no markdown to keep Codio happy)
table_columns = [{"name": "", "id": "logo"}]
for c in cols_for_table:
    if c == "logo":
        continue
    table_columns.append({"name": c, "id": c, "deletable": False, "selectable": True})

# Compact logo style for the first column (smaller and clearer)
logo_col_style = []
if logo_uri:
    logo_col_style = [{
        "if": {"column_id": "logo"},
        "backgroundImage": f'url("{logo_uri}")',
        "backgroundRepeat": "no-repeat",
        "backgroundPosition": "center",
        "backgroundSize": "26px 12px",   # smaller image inside the cell
        "color": "rgba(0,0,0,0)",        # hide any text
        "textShadow": "none",
        "width": "38px", "minWidth": "38px", "maxWidth": "38px",
        "paddingLeft": "0", "paddingRight": "0",
    }]

app.layout = html.Div([
    header,
    html.Br(),

    html.Div([
        html.Label("Choose a filter"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset (All Records)',             'value': 'reset'},
                {'label': 'Water Rescue',                    'value': 'water'},
                {'label': 'Mountain or Wilderness',          'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            ],
            value='reset',
            labelStyle={'display': 'block'}
        )
    ], style={'width': '260px', 'padding': '10px'}),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=table_columns,
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'minWidth': '90px', 'width': '120px', 'maxWidth': '260px'},
        style_header={'fontWeight': 'bold'},
        style_data={'height': '26px'},          # compact row height matches icon size
        style_data_conditional=logo_col_style
    ),

    html.Br(),

    html.Div(style={'display': 'flex'}, children=[
        html.Div(id='graph-id', style={'flex': '1', 'padding': '10px'}),
        html.Div(id='map-id',   style={'flex': '1', 'padding': '10px'}),
    ])
])


# -----------------------
# Controller / Callbacks
# -----------------------
def build_filter_query(tag: str):
    """Filters aligned to rubric; treat 'adult' as containing 'year'; intact can be intake or outcome."""
    adult_age = {"$regex": r"year", "$options": "i"}
    intact_clause = {"$or": [
        {"sex_upon_outcome": {"$regex": "Intact", "$options": "i"}},
        {"sex_upon_intake":  {"$regex": "Intact", "$options": "i"}},
    ]}

    if tag == "water":
        return {
            "animal_type": "Dog",
            **intact_clause,
            "age_upon_outcome": adult_age,
            "breed": {"$in": [
                "Labrador Retriever", "Labrador Retriever Mix",
                "Chesapeake Bay Retriever", "Chesapeake Bay Retriever Mix",
                "Golden Retriever", "Golden Retriever Mix"]},
        }
    if tag == "mountain":
        return {
            "animal_type": "Dog",
            **intact_clause,
            "age_upon_outcome": adult_age,
            "breed": {"$in": [
                "German Shepherd", "German Shepherd Mix",
                "Doberman Pinscher", "Doberman Pinscher Mix",
                "Rottweiler", "Rottweiler Mix"]},
        }
    if tag == "disaster":
        return {
            "animal_type": "Dog",
            **intact_clause,
            "age_upon_outcome": adult_age,
            "breed": {"$in": [
                "German Shepherd", "German Shepherd Mix",
                "Belgian Malinois", "Belgian Malinois Mix",
                "Bloodhound", "Bloodhound Mix",
                "Doberman Pinscher", "Doberman Pinscher Mix",
                "Rottweiler", "Rottweiler Mix"]},
        }
    return {}

@app.callback(Output('datatable-id', 'data'),
              Input('filter-type', 'value'))
def update_table(filter_value):
    q = build_filter_query(filter_value)
    view_df = fetch_df(q, limit=2000)
    if view_df.empty:
        view_df = df.copy()
    return view_df.to_dict('records')

@app.callback(Output('graph-id', 'children'),
              Input('datatable-id', 'derived_virtual_data'))
def update_chart(viewData):
    cur = pd.DataFrame(viewData) if viewData is not None else df.copy()
    if 'logo' in cur.columns:
        cur = cur.drop(columns=['logo'])
    if cur.empty or 'breed' not in cur.columns:
        return html.Div("No data to chart.")
    counts = cur['breed'].value_counts().reset_index()
    counts.columns = ['breed', 'count']
    fig = px.bar(counts.head(15), x='breed', y='count', title='Top Breeds in View')
    fig.update_layout(xaxis_title='Breed', yaxis_title='Count', margin=dict(l=10, r=10, t=40, b=10))
    return dcc.Graph(figure=fig)

@app.callback(Output('map-id', 'children'),
              Input('datatable-id', 'derived_virtual_data'),
              Input('datatable-id', 'derived_virtual_selected_rows'))
def update_map(viewData, idx):
    if viewData is None:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return []
    if 'logo' in dff.columns:
        dff = dff.drop(columns=['logo'])

    row = 0 if not idx else idx[0]
    # Many rows store lat/lon at fixed positions; fallback to Austin if missing
    try:
        lat = float(dff.iloc[row, 13]); lon = float(dff.iloc[row, 14])
    except Exception:
        lat = float(dff.get('location_lat',  pd.Series([30.75]))[row])
        lon = float(dff.get('location_long', pd.Series([-97.48]))[row])
    name  = str(dff.iloc[row]['name'])  if 'name'  in dff.columns else 'Animal'
    breed = str(dff.iloc[row]['breed']) if 'breed' in dff.columns else ''
    return [dl.Map(style={'width': '100%', 'height': '500px'},
                   center=[lat, lon], zoom=10,
                   children=[dl.TileLayer(id='base-layer-id'),
                             dl.Marker(position=[lat, lon], children=[
                                 dl.Tooltip(name),
                                 dl.Popup([html.H1("Animal Name"), html.P(name), html.P(breed)])
                             ])])]


# -----------------------
# Run (inline via proxy; fresh port to avoid collisions)
# -----------------------
app.run_server(mode='inline', debug=False, port=8051)


Initial rows loaded: 1000


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Oct/2025 20:23:47] "GET /_alive_c2861978-19af-4b8b-8336-696ac7131149 HTTP/1.1" 200 -


127.0.0.1 - - [18/Oct/2025 20:23:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2025 20:23:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2025 20:23:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2025 20:23:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2025 20:23:47] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Oct/2025 20:23:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2025 20:23:47] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Oct/2025 20:23:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2025 20:23:47] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Oct/2025 20:23:47] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Oct/2025 20:23:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - -